In [24]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service
from urllib.request import Request, urlopen
from user_agent import generate_user_agent
from bs4 import BeautifulSoup
import time
from selenium.webdriver.common.keys import Keys
import pandas as pd
from time import sleep


################################################
##################스타벅스 데이터##################
################################################


driver = webdriver.Chrome(service=Service("../driver/chromedriver"))
driver.maximize_window()
driver.get("https://www.starbucks.co.kr/menu/drink_view.do?product_cd=110563")

url = "https://www.starbucks.co.kr/menu/drink_view.do?product_cd=110563"
req = Request(url, headers={"User-Agent": generate_user_agent()})
html = urlopen(req)
soup = BeautifulSoup(html, "html.parser")

Menu_name = ['아메리카노','카페라떼','카페모카']
Kcal = []
Caffeine = []
records = []

# # 1) 제품 영양 정보 블록
info = soup.select_one("div.product_view_info div.product_info_content")
if not info:
    raise RuntimeError("영양정보 컨테이너를 찾을 수 없습니다")

# # 2) 칼로리(li.kcal) & 카페인(li.caffeine) 태그
kcal_tag   = info.find("li", class_="kcal")
caff_tag   = info.find_all("li")

# 아메리카노
kcal = driver.find_element(By.XPATH, '''//*[@id="container"]/div[2]/div[1]/div[2]/form/fieldset/div/div[2]/ul[1]/li[1]/dl/dd''')
caffeine = driver.find_element(By.XPATH, '''//*[@id="container"]/div[2]/div[1]/div[2]/form/fieldset/div/div[2]/ul[2]/li[3]/dl/dd''')
Kcal.append(kcal.text)
Caffeine.append(caffeine.text)

#카페라떼 
driver.get("https://www.starbucks.co.kr/menu/drink_view.do?product_cd=41")

kcal = driver.find_element(By.XPATH, '''//*[@id="container"]/div[2]/div[1]/div[2]/form/fieldset/div/div[2]/ul[1]/li[1]/dl/dd''')
caffeine = driver.find_element(By.XPATH, '''//*[@id="container"]/div[2]/div[1]/div[2]/form/fieldset/div/div[2]/ul[2]/li[3]/dl/dd''')
Kcal.append(kcal.text)
Caffeine.append(caffeine.text)


#카페모카
driver.get("https://www.starbucks.co.kr/menu/drink_view.do?product_cd=110566")

kcal = driver.find_element(By.XPATH, '''//*[@id="container"]/div[2]/div[1]/div[2]/form/fieldset/div/div[2]/ul[1]/li[1]/dl/dd''')
caffeine = driver.find_element(By.XPATH, '''//*[@id="container"]/div[2]/div[1]/div[2]/form/fieldset/div/div[2]/ul[2]/li[3]/dl/dd''')
Kcal.append(kcal.text)
Caffeine.append(caffeine.text)

Starbucks = pd.DataFrame({
    "Brand": ["스타벅스"] * len(Menu_name),
    "Menu_name": Menu_name,
    "Kcal": Kcal,
    "Caffeine": Caffeine
})
driver.quit()

################################################
##################할리스 데이터####################
################################################

# 웹드라이버 실행
driver = webdriver.Chrome(service=Service("../driver/chromedriver"))
driver.maximize_window()
driver.get("https://www.hollys.co.kr/menu/espresso.do")

try:
    wait = WebDriverWait(driver, 10)

    # id="more" 인 요소를 대상으로 클릭
    more_button = wait.until(EC.element_to_be_clickable((By.ID, "more")))

    # 스크롤하고 클릭
    actions = ActionChains(driver)
    actions.move_to_element(more_button).perform()
    more_button.click()

    #print("MORE 버튼 클릭 성공")

except Exception as e:
    print("MORE 버튼 클릭 실패:", e)

#스크롤 맨 아래로 이동 (모든 메뉴 로드되게)
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(1)

# 아메리카노 클릭
menu_names = driver.find_element(By.CLASS_NAME, "menu_list01.mar_t_40")
menu_names = menu_names.find_elements(By.TAG_NAME, "a")

# for menu_name in menu_names:
#     print(menu_name.text)
# print("=================================")
coffe_list = ["아메리카노","카페 라떼 ","카페 모카"]
records = []

for coffee in coffe_list:
    for item in menu_names:
        if coffee in item.text:
            item.click()


    #스크롤 올리기
    driver.execute_script("window.scrollTo(0, 0);")

    # BeautifulSoup 파싱
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    #driver.quit()

    # 영양정보 추출

    table = soup.find("table", summary=coffee)

    iced_row = table.find("tbody")
    #print(table)

    if iced_row:
        td_tags = iced_row.find_all("tr")
        td_tags = td_tags[1].find_all("td")
        for td in td_tags:
            text = td.get_text(strip=True)
            if "mg" in text:
                caffeine = text
            elif "kcal" in text:
                kcal = text

    #print(f"{coffee}: {caffeine}, {kcal}")

    records.append({
        "Menu_name": coffee,
        "Kcal": kcal,
        "Caffeine": caffeine
    })

Hollys= pd.DataFrame(records)
Hollys.insert(0, "Brand", ["할리스"] * len(Hollys))
driver.quit()


################################################
##################이디야 데이터####################
################################################

# 크롬 드라이버 실행 및 창 최대화
driver = webdriver.Chrome(service=Service("../driver/chromedriver"))
driver.maximize_window()
wait = WebDriverWait(driver, 10)

# 이디야 음료 페이지 열기
driver.get("https://www.ediya.com/contents/drink.html")


# 1. 아메리카노 검색 및 추출

search_input = wait.until(EC.presence_of_element_located((By.ID, "skeyword")))
search_input.clear()
search_input.send_keys("아메리카노")

search_button = driver.find_element(By.CSS_SELECTOR, "div.search_bar button")
driver.execute_script("arguments[0].click();", search_button)

nutri_link = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "a[onclick=\"show_nutri('178')\"]")))
driver.execute_script("arguments[0].click();", nutri_link)

wait.until(EC.presence_of_element_located((By.ID, "nutri_178")))
time.sleep(1)
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(0.5)

calorie_ame = driver.find_element(By.XPATH, """//*[@id="nutri_178"]/div[3]/div[1]/dl[1]/dd""").text.strip().replace("(", "").replace(")", "")
caffeine_ame = driver.find_element(By.XPATH, """//*[@id="nutri_178"]/div[3]/div[1]/dl[6]/dd""").text.strip().replace("(", "").replace(")", "")
print(f"[아메리카노] 칼로리: {calorie_ame} / 카페인: {caffeine_ame}")


# 2. 카페라떼 검색 및 추출


search_input = wait.until(EC.presence_of_element_located((By.ID, "skeyword")))
search_input.clear()
search_input.send_keys("카페 라떼")

search_button = driver.find_element(By.CSS_SELECTOR, "div.search_bar button")
driver.execute_script("arguments[0].click();", search_button)

time.sleep(1)
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(0.5)

more_btn = wait.until(EC.presence_of_element_located((By.XPATH, "//*[@id='contentWrap']/div[5]/div/div/a")))
driver.execute_script("arguments[0].click();", more_btn)

time.sleep(1)
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(0.5)

nutri_link = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "a[onclick=\"show_nutri('174')\"]")))
driver.execute_script("arguments[0].click();", nutri_link)

wait.until(EC.presence_of_element_located((By.ID, "nutri_174")))
time.sleep(1)

calorie_lat = driver.find_element(By.XPATH, """//*[@id="nutri_174"]/div[2]/div[1]/dl[1]/dd""").text.strip().replace("(", "").replace(")", "")
caffeine_lat = driver.find_element(By.XPATH, """//*[@id="nutri_174"]/div[2]/div[1]/dl[6]/dd""").text.strip().replace("(", "").replace(")", "")
print(f"[카페라떼] 칼로리: {calorie_lat} / 카페인: {caffeine_lat}")


# 3. 카페모카 검색 및 추출


search_input = wait.until(EC.presence_of_element_located((By.ID, "skeyword")))
search_input.clear()
search_input.send_keys("카페 모카")

search_button = driver.find_element(By.CSS_SELECTOR, "div.search_bar button")
driver.execute_script("arguments[0].click();", search_button)

time.sleep(1)
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(0.5)

mocha_link = wait.until(EC.presence_of_element_located((By.XPATH, "//*[@id='menu_ul']/li[6]/a")))
driver.execute_script("arguments[0].click();", mocha_link)

wait.until(EC.presence_of_element_located((By.ID, "nutri_168")))
time.sleep(1)
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(0.5)

calorie_moc = driver.find_element(By.XPATH, """//*[@id="nutri_168"]/div[3]/div[1]/dl[1]/dd""").text.strip().replace("(", "").replace(")", "")
caffeine_moc = driver.find_element(By.XPATH, """//*[@id="nutri_168"]/div[3]/div[1]/dl[6]/dd""").text.strip().replace("(", "").replace(")", "")
print(f"[카페모카] 칼로리: {calorie_moc} / 카페인: {caffeine_moc}")


Ediya = pd.DataFrame({
    "Menu_name": ["아메리카노", "카페라떼", "카페모카"],
    "Kcal": [calorie_ame, calorie_lat, calorie_moc],
    "Caffeine": [caffeine_ame, caffeine_lat, caffeine_moc]
})
Ediya.insert(0, "Brand", ["이디야"] * len(Ediya))

driver.quit()

######################################################################
def Page_Open(Name):
    Page_link_num = driver.find_element(By.TAG_NAME,"nav")

    Page_run = Page_link_num.find_element(By.PARTIAL_LINK_TEXT, Name)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    sleep(0.5)
    Page_run.click()
    
# 기존 탭 저장
    original_window = driver.current_window_handle

# 현재 열려있는 모든 탭 핸들 가져오기
    windows = driver.window_handles

    return original_window, windows


def Cafe_Open(Name):
    Cafe = driver.find_element(By.PARTIAL_LINK_TEXT, Name)
    driver.execute_script("arguments[0].scrollIntoView(false);", Cafe)
    Cafe.click()
    
# 기존 탭 저장
    original_window = driver.current_window_handle

# 현재 열려있는 모든 탭 핸들 가져오기
    windows = driver.window_handles

# 새로 열린 탭으로 전환
    #driver.switch_to.window(windows[-1])
    #print("새 탭 타이틀:", driver.title)

    return original_window, windows

coffee_name = [
    '''//*[@id="contents"]/div/div/ul/li[2]''',
    '''//*[@id="contents"]/div/div/ul/li[8]''',
    '''//*[@id="contents"]/div/div/ul/li[9]'''    
]

def C_Bean(X_path):
    info_M = driver.find_element(By.XPATH, X_path)
    info_name = info_M.find_element(By.CLASS_NAME, "kor")
    info_list = info_M.find_element(By.CLASS_NAME, "info")
    info_list = info_list.find_elements(By.TAG_NAME, "dl")
    driver.execute_script("arguments[0].scrollIntoView(false);", info_M)
    
    return info_name, info_list

##############################################################


################################################
##################커피빈 데이터####################
################################################

#커피빈 커피 수집
Coffee_N = []
Kcal = []
Caffeine = []
CoffeeBean = pd.DataFrame()

driver = webdriver.Chrome(service=Service("../driver/chromedriver"))
driver.maximize_window()
driver.get("https://www.coffeebeankorea.com/menu/list.asp?category=13")
sleep(0.5)

driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")

#키워드 검색
Page_link = driver.find_element(By.CLASS_NAME, "paging")
Page_link = Page_link.find_element(By.LINK_TEXT, "1")
Page_link.click()
sleep(1)

for idx, Name in enumerate(coffee_name):
    if len(Coffee_N) == 1:
        Page_link = driver.find_element(By.CLASS_NAME, "paging")
        Page_link = Page_link.find_element(By.LINK_TEXT, "2")
        Page_link.click()
    
    info_name, info_list = C_Bean(Name)

    sleep(1)
    Coffee_N.append(info_name.text)
    for name in info_list:
        C_info = name.text.split('\n')[1]
        C_num = name.text.split('\n')[0] + " " + name.text.split('\n')[2]

        if C_info == "열량":
            Kcal.append(C_num)

        if C_info == "카페인":
            Caffeine.append(C_num)

    sleep(1)

CoffeeBean = pd.DataFrame({
    "Menu_name": Coffee_N,
    "Kcal": Kcal,
    "Caffeine": Caffeine
})

CoffeeBean.insert(0, "Brand", ["커피빈"] * len(CoffeeBean))

driver.quit()

################################################
##################메가 데이터#####################
################################################

#MEGA 커피 수집
Coffee_N = []
Kcal = []
Caffeine = []
Mega = pd.DataFrame()

driver = webdriver.Chrome(service=Service("../driver/chromedriver"))
driver.maximize_window()
driver.get("https://www.mega-mgccoffee.com/menu/?menu_category1=1&menu_category2=1")
sleep(0.5)

info_M = driver.find_element(By.XPATH, '''/html/body/div[4]/div[3]/div/div[3]/div[2]/div[3]''')
driver.execute_script("arguments[0].scrollIntoView(false);", info_M)
checkBox = info_M.find_elements(By.CLASS_NAME, "checkbox_wrap.list_checkbox")
sleep(0.5)
for check in checkBox:
    if check.text == "커피":
        check.click()
        break

sleep(0.5)
driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")

move_page = driver.find_element(By.CLASS_NAME, "board_page")
move_page = move_page.find_elements(By.TAG_NAME, "li")

move_page[2].click()
sleep(0.5)

Menu_list = driver.find_element(By.CLASS_NAME, "cont_gallery_list.cont_gallery_list2.cont_list.cont_list4.cont_list_m.cont_list_m2.wfull")
Menu_list = Menu_list.find_elements(By.TAG_NAME, "li")
sleep(1)

#66 - 카페라떼 / 72-카페모카 / 114 - 아메리카노
C_Menu = [Menu_list[66], Menu_list[72], Menu_list[114]]

for C_menu_list in C_Menu:
    driver.execute_script("arguments[0].scrollIntoView(false);", C_menu_list)
    C_menu_list.click()

    C_menu_list2 = C_menu_list.find_element(By.CLASS_NAME, "inner_modal")

    Coffee_name = C_menu_list2.find_element(By.CLASS_NAME, "cont_text_inner.cont_text_title")
    Coffee_N.append(Coffee_name.text)

    Coffee_Kcal = C_menu_list2.find_elements(By.CLASS_NAME, "cont_text_inner")
    Kcal.append(Coffee_Kcal[3].text.split("제공량 ")[1])


    Coffee_Caffeine = C_Menu[0].find_element(By.CLASS_NAME, "cont_list.cont_list2.cont_list_small.cont_list_small2")
    Coffee_Caffeine = Coffee_Caffeine.find_elements(By.TAG_NAME, "li")
    Caffeine.append(Coffee_Caffeine[4].text.split(" ")[1])

Mega = pd.DataFrame({
    "Menu_name": Coffee_N,
    "Kcal": Kcal,
    "Caffeine": Caffeine
})

Mega.insert(0, "Brand", ["메가"] * len(Mega))

driver.quit()

################################################
##################더벤티 데이터####################
################################################

#더벤티
Coffee_N = []
Kcal = []
Caffeine = []
Theventi = pd.DataFrame()

driver = webdriver.Chrome(service=Service("../driver/chromedriver"))
driver.maximize_window()
driver.get("https://www.theventi.co.kr/new2022/menu/index.html")
sleep(0.5)

M_list = driver.find_element(By.CLASS_NAME, "sub-nav__wrap.sub-nav-clone--depth2")
driver.execute_script("arguments[0].scrollIntoView(false);", M_list)
M_list = M_list.find_element(By.LINK_TEXT, "전체메뉴")
sleep(0.5)

M_list.click()
sleep(0.5)

M_kind = driver.find_element(By.CLASS_NAME, "tab.box")
M_kind = M_kind.find_elements(By.TAG_NAME, "li")
M_kind[1].click()
sleep(0.5)

Menu_list = driver.find_element(By.CLASS_NAME, "menu_list")
Menu_list = Menu_list.find_elements(By.TAG_NAME, "a")
sleep(0.5)

# 1 = 아이스 아메리카노, 2 = 카페라떼, 10 = 카페모카
C_Menu = [Menu_list[1], Menu_list[2], Menu_list[10]]

for C_Menu_name in C_Menu:
    driver.execute_script("arguments[0].scrollIntoView(false);", C_Menu_name)
    C_Menu_name.click()
    sleep(0.5)

    C_Menu_list = driver.find_element(By.CLASS_NAME, "mfp-content")
    C_Menu_list2 = C_Menu_list.find_element(By.TAG_NAME, "p")
    C_Menu_list2 = C_Menu_list2.find_elements(By.TAG_NAME, "span")

    Coffee_N.append(C_Menu_list2[2].text.replace(" (벤티/더벤티)", ""))


    C_Menu_list1 = C_Menu_list.find_element(By.TAG_NAME, "tbody")
    C_Menu_list1 = C_Menu_list1.find_elements(By.TAG_NAME, "td")

#1 = 열량, 6 = 카페인
    Kcal.append(C_Menu_list1[1].text.replace(" (벤티/더벤티)", "") + "Kcal")
    Caffeine.append(C_Menu_list1[6].text.replace(" (고카페인)", "") + "mg")

    Close = C_Menu_list.find_element(By.CLASS_NAME, "xi-close")
    Close.click()
    sleep(0.5)

Theventi = pd.DataFrame({
    "Menu_name": Coffee_N,
    "Kcal": Kcal,
    "Caffeine": Caffeine
})

Theventi.insert(0, "Brand", ["더벤티"] * len(Theventi))

driver.quit()

################################################
##################컴포즈 데이터####################
################################################

#컴포즈
Coffee_N = []
Kcal = []
Caffeine = []
compose = pd.DataFrame()

driver = webdriver.Chrome(service=Service("../driver/chromedriver"))
driver.maximize_window()
driver.get("https://composecoffee.com/menu")
sleep(3)

# 모바일 메뉴 버튼
menu_button = driver.find_element(By.CLASS_NAME, "nav-item.dropdown.d-block.d-lg-none")
driver.execute_script("arguments[0].click();", menu_button)
sleep(1)

# 메뉴가 펼쳐진 후 '커피 · 더치' 클릭
coffee_menu = driver.find_element(By.LINK_TEXT, "커피 · 더치")
driver.execute_script("arguments[0].click();", coffee_menu)
sleep(1)

# 메뉴 리스트 추출
Menu_list = driver.find_elements(By.CLASS_NAME, "col-lg-3.col-md-4.col-sm-6.itemBox")
Menu_list_N = []

actions = ActionChains(driver)
for name in Menu_list:
    driver.execute_script("arguments[0].scrollIntoView(false);", name)
    sleep(0.5)

    if "ICE 아메리카노" in name.text:
        actions.move_to_element(name).perform()
        Menu_list_N.append(name.text.split("\n"))
        sleep(0.5)
    elif "ICE 카페라떼" in name.text:
        actions.move_to_element(name).perform()
        Menu_list_N.append(name.text.split("\n"))
        sleep(0.5)
    elif "ICE 카페모카" in name.text:
        actions.move_to_element(name).perform()
        Menu_list_N.append(name.text.split("\n"))
        sleep(0.5)

# 0 = name / 4 = Kcal / 2 = 카페인
for name in Menu_list_N:
    Coffee_N.append(name[0])
    Kcal.append((name[4].split(" "))[3] + "Kcal")
    Caffeine.append((name[2].split(" ")[5]).split("/")[0])

compose = pd.DataFrame({
    "Menu_name": Coffee_N,
    "Kcal": Kcal,
    "Caffeine": Caffeine
})

compose.insert(0, "Brand", ["컴포즈"] * len(compose))

driver.quit()


################################################
##################빽다방 데이터####################
################################################

Coffee_N = []
Kcal = []
Caffeine = []
Paikdabang = pd.DataFrame()

driver = webdriver.Chrome(service=Service("../driver/chromedriver"))
driver.maximize_window()
driver.get("https://paikdabang.com/menu/menu_new/")
sleep(0.5)

M_list = driver.find_element(By.CLASS_NAME, "page_tab.st-2")
M_list = M_list.find_element(By.LINK_TEXT, "커피")
M_list.click()
sleep(0.5)

Menu_list = driver.find_element(By.CLASS_NAME, "menu_list.clear")
Menu_list = Menu_list.find_elements(By.CLASS_NAME, "li")

Menu_list = driver.find_element(By.CLASS_NAME, "menu_list.clear")
driver.execute_script("arguments[0].scrollIntoView(false);", Menu_list)
sleep(0.5)

Menu_list = Menu_list.find_elements(By.TAG_NAME, "li")

Menu_list_N = []

for idx, M_name in enumerate(Menu_list):

    if "아메리카노(ICED)" == M_name.text:
        driver.execute_script("arguments[0].scrollIntoView(false);", M_name)
        sleep(1)
        M_name.click()
        sleep(0.5)

        Coffee_N.append(M_name.text.split("\n")[0].replace("(ICED)", ""))
        Kcal.append(M_name.text.split("\n")[8]+ "Kcal")
        Caffeine.append(M_name.text.split("\n")[6]+"mg")
        sleep(0.5)

        Menu_list_N.append(M_name.text)

    elif "카페라떼(ICED)" == M_name.text:
        driver.execute_script("arguments[0].scrollIntoView(false);", M_name)
        sleep(1)
        M_name.click()
        sleep(0.5)

        Coffee_N.append(M_name.text.split("\n")[0].replace("(ICED)", ""))
        Kcal.append(M_name.text.split("\n")[9]+ "Kcal")
        Caffeine.append(M_name.text.split("\n")[7]+"mg")
        sleep(0.5)

        Menu_list_N.append(M_name.text)

    elif "카페모카(ICED)" == M_name.text:
        driver.execute_script("arguments[0].scrollIntoView(false);", M_name)
        sleep(1)
        M_name.click()
        sleep(0.5)

        Coffee_N.append(M_name.text.split("\n")[0].replace("(ICED)", ""))
        Kcal.append(M_name.text.split("\n")[10]+ "Kcal")
        Caffeine.append(M_name.text.split("\n")[8]+"mg")
        sleep(0.5)

        Menu_list_N.append(M_name.text)

Paikdabang = pd.DataFrame({
    "Menu_name": Coffee_N,
    "Kcal": Kcal,
    "Caffeine": Caffeine
})

Paikdabang.insert(0, "Brand", ["빽다방"] * len(Paikdabang))

driver.quit()

################################################
################탐앤탐스 데이터####################
################################################

# 탐앤탐스

driver = webdriver.Chrome(service=Service("../driver/chromedriver"))
driver.maximize_window()
driver.get("https://www.tomntoms.com/menu/drink")
sleep(1)

search = driver.find_element(By.XPATH, """//*[@id="root"]/section[4]/ul/li[6]/div/label""")
search.click()
Menu_name = []
Kcal = []
Caffeine = []

#카페라떼 
search = driver.find_elements(By.CLASS_NAME, "relative.w-full")

# 1 = 아이스아메리카노, 2 = 아이스 카페라떼, 8 = 아이스 카페모카
menu = [1, 2]
list_num = [0, 11, 21]
for menu_name in menu:
    search[menu_name].click()
    sleep(1)

#l_list = search[2].find_elements(By.TAG_NAME, "div")

    name = search[menu_name].text.split('\n')

    for idx, l_ in enumerate(list_num):
        print(str(idx) + ". " + name[l_])
        if str(idx) == '0':
            Menu_name.append(name[l_])
        elif str(idx) == '1':
            Kcal.append(name[l_])
        else:
            Caffeine.append(name[l_])

# 8번 카페모카 스크롤 내려서 클릭
element = driver.find_element(By.XPATH, '''//*[@id="root"]/section[5]/div[1]/div[9]/div[1]/div''')
driver.execute_script("arguments[0].scrollIntoView(false);", element)
search = driver.find_element(By.XPATH, '''//*[@id="root"]/section[5]/div[1]/div[9]''')
sleep(1)
search.click()


l_list = search.find_element(By.TAG_NAME, "div")
name = l_list.text.split('\n')


for idx, l_ in enumerate(list_num):
    print(str(idx) + ". " + name[l_])
    if str(idx) == '0':
        Menu_name.append(name[l_])
    elif str(idx) == '1':
        Kcal.append(name[l_])
    else:
        Caffeine.append(name[l_])


import pandas as pd

records = {"Menu_name: ": Menu_name,
        "Kcal": Kcal,
        "Caffeine": Caffeine}

Tomntoms= pd.DataFrame(records)

Tomntoms.insert(0, "Brand", ["탐앤탐스"] * len(Tomntoms))

driver.quit()

# 컬럼명 통일
Starbucks.columns = ["Brand","menu", "Kcal", "Caffeine"]
Hollys.columns = ["Brand","menu", "Kcal", "Caffeine"]
Ediya.columns = ["Brand","menu", "Kcal", "Caffeine"]
CoffeeBean.columns = ["Brand","menu", "Kcal", "Caffeine"]
Mega.columns = ["Brand","menu", "Kcal", "Caffeine"]
Theventi.columns = ["Brand","menu", "Kcal", "Caffeine"]
compose.columns = ["Brand","menu", "Kcal", "Caffeine"]
Paikdabang.columns = ["Brand","menu", "Kcal", "Caffeine"]
Tomntoms.columns = ["Brand","menu", "Kcal", "Caffeine"]  # 원래 "Menu_name: "이라 바꿔야 함

# 합치기
total_df = pd.concat([
    Starbucks,
    Hollys,
    Ediya,
    CoffeeBean,
    Mega,
    Theventi,
    compose,
    Paikdabang,
    Tomntoms
], ignore_index=True)

# 결과 확인
print(total_df)

# CSV 저장 원하면 아래 주석 해제
# total_df.to_csv("total_coffee_info.csv", index=False, encoding="utf-8-sig")

# 1. menu 통일 (포함 여부로 통일)
def unify_menu(name):
    if "아메리카노" in name:
        return "아메리카노"
    elif "카페라떼" in name or "카페 라떼" in name:
        return "카페라떼"
    elif "카페모카" in name:
        return "카페모카"
    else:
        return name  # 혹시 모르니 다른 값은 그대로 두기

total_df["menu"] = total_df["menu"].apply(unify_menu)

# 2. Kcal 단위 제거 후 int 변환
total_df["Kcal"] = total_df["Kcal"].str.replace(r"[^\d\.]", "", regex=True).astype(float).astype(int)

# 3. Caffeine 단위 제거 후 int 변환
total_df["Caffeine"] = total_df["Caffeine"].str.replace(r"[^\d\.]", "", regex=True).astype(float).astype(int)

# 결과 확인
print(total_df)







[아메리카노] 칼로리: 16kcal / 카페인: 158mg
[카페라떼] 칼로리: 142kcal / 카페인: 158mg
[카페모카] 칼로리: 353kcal / 카페인: 171mg
0. 아이스 아메리카노
1. 5
2. 97.95mg
0. 아이스 카페라떼
1. 107
2. 97.95mg
0. 아이스 카페모카
1. 242
2. 97.95mg
   Brand       menu       Kcal Caffeine
0   스타벅스      아메리카노         10      150
1   스타벅스       카페라떼        180       75
2   스타벅스       카페모카        250       95
3    할리스      아메리카노     12kcal    114mg
4    할리스     카페 라떼      96kcal    127mg
5    할리스      카페 모카    237kcal    132mg
6    이디야      아메리카노     16kcal    158mg
7    이디야       카페라떼    142kcal    158mg
8    이디야       카페모카    353kcal    171mg
9    커피빈       카페모카   305 Kcal   102 mg
10   커피빈  아이스 아메리카노     5 Kcal    91 mg
11   커피빈   아이스 카페라떼   119 Kcal    91 mg
12    메가       카페라떼  145.1kcal  199.1mg
13    메가       카페모카   294.4cal  199.1mg
14    메가      아메리카노   12.2kcal  199.1mg
15   더벤티  아이스 아메리카노     11Kcal    192mg
16   더벤티       카페라떼    160Kcal    296mg
17   더벤티       카페모카    423Kcal    302mg
18   컴포즈  ICE 아메리카노     15Kcal    156mg
19   컴포즈   I

In [25]:
driver.quit()

In [26]:
total_df

,Brand,menu,Kcal,Caffeine
0,스타벅스,아메리카노,10,150
1,스타벅스,카페라떼,180,75
2,스타벅스,카페모카,250,95
3,할리스,아메리카노,12,114
4,할리스,카페라떼,96,127
5,할리스,카페 모카,237,132
6,이디야,아메리카노,16,158
7,이디야,카페라떼,142,158
8,이디야,카페모카,353,171
9,커피빈,카페모카,305,102
